In [1]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

### Load the Tweets data

In [2]:
data = pd.read_csv('dataset/twitter_sentiments.csv')
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [8]:
data.shape

(31962, 3)

In [9]:
data.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

### Split into train and test

In [4]:
train, test = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=21)

In [5]:
train.shape, test.shape

((25569, 3), (6393, 3))

In [6]:
train.label.value_counts(normalize=True)

0    0.929837
1    0.070163
Name: label, dtype: float64

In [7]:
test.label.value_counts(normalize=True)

0    0.929923
1    0.070077
Name: label, dtype: float64

### Create TF-IDF vector of all Tweets

In [11]:
# Create a TF-IDF vectorizer object
tfidf_vectorizer = TfidfVectorizer(lowercase=True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)

# Fit the object with the training data tweets
tfidf_vectorizer.fit(train.tweet)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, sublinear_tf=False,
                token_pa

In [12]:
# Transform the train and test data
train_idf = tfidf_vectorizer.transform(train.tweet)
test_idf = tfidf_vectorizer.transform(test.tweet)

In [16]:
len(train_idf.data)

105309

### Logistic Regression

In [17]:
# Create the LogisticRegression Model
model_LR = LogisticRegression()

In [18]:
# Fit the model with the training data
model_LR.fit(train_idf, train.label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
# Predict the label on the training data
predict_train = model_LR.predict(train_idf)

In [23]:
# Predict the label on the testing data
predict_test = model_LR.predict(test_idf)

In [24]:
# F1 score on train data
f1_score(y_true=train.label, y_pred=predict_train)

0.4888178913738019

In [25]:
# F1 score on test data
f1_score(y_true=test.label, y_pred=predict_test)

0.45751633986928114

### Define the Pipeline

In [26]:
# Define the stages of the pipeline
pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=True,
                                                    max_features=1000,
                                                    stop_words=ENGLISH_STOP_WORDS)),
                           ('model', LogisticRegression())])

In [27]:
# Fit the pipeline model with training data
pipeline.fit(train.tweet, train.label)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=1000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterward...
                                 strip_accents=None, sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),


In [49]:
# Test the pipeline with a sample tweet
text = ["Michael Jordan is great"]

# Predict the label using the pipeline
pipeline.predict(text)[0]

0

### Save the pipeline

In [50]:
from joblib import dump

# Dump the pipelinoe model
dump(pipeline, filename='text_classification.joblib')

['text_classification.joblib']